<a href="https://colab.research.google.com/github/samuelacquatey/DS-study-notebooks/blob/main/ALX_FINAL_EXAM_Movies_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from sqlalchemy import create_engine, inspect, text
import pandas as pd

data = create_engine('sqlite:///TMDB.db')

connection = data.connect()

inspector = inspect(data)

columns = inspector.get_columns('movies')

colums_names = [column['name'] for column in columns]

print(colums_names)



['movie_id', 'title', 'release_date', 'budget', 'homepage', 'original_language', 'original_title', 'overview', 'popularity', 'revenue', 'runtime', 'release_status', 'tagline', 'vote_average', 'vote_count']


In [14]:
foreign_keys = inspector.get_foreign_keys('languagemap')

no_fk = 0
for key in foreign_keys:
    no_fk += 1
print(no_fk)


2


In [15]:
with data.connect() as connection:
  create_view_sql = text("""
  CREATE VIEW Not_Released AS
  SELECT * FROM movies
  WHERE release_status <> 'Released';
  """)

  # Execute the SQL command
  connection.execute(create_view_sql)

  print("View 'Not_Released' created successfully.")


OperationalError: (sqlite3.OperationalError) table Not_Released already exists
[SQL: 
  CREATE VIEW Not_Released AS 
  SELECT * FROM movies 
  WHERE release_status <> 'Released';
  ]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [16]:
with data.connect() as connection:
  query = text("""
  SELECT title, release_date, release_status
  FROM Not_Released;
  """)

  # Execute the query and fetch the results into a DataFrame
  result = connection.execute(query)
  df = pd.DataFrame(result.fetchall(), columns=result.keys())

  # Display the DataFrame
  print(df)

                      title                release_date   release_status
0            Little Big Top  2006-01-01 00:00:00.000000          Rumored
1       The Helix... Loaded  2005-01-01 00:00:00.000000          Rumored
2             Higher Ground  2011-08-26 00:00:00.000000  Post Production
3      Crying with Laughter  2009-06-01 00:00:00.000000          Rumored
4  The Harvest (La Cosecha)  2011-07-29 00:00:00.000000          Rumored
5             The Naked Ape  2006-09-16 00:00:00.000000          Rumored
6            Brotherly Love  2015-04-24 00:00:00.000000  Post Production
7           Dancin' It's On  2015-10-16 00:00:00.000000  Post Production


In [17]:
with data.connect() as connection:
  not_original = text("""
  SELECT COUNT(*)
  FROM movies
  WHERE original_title <> title;
  """)

  result = connection.execute(not_original)
  print(result.fetchall())

[(261,)]


In [18]:
with data.connect() as connection:
  most_popular = text("""
  SELECT title, popularity, release_date, budget
  FROM movies
  WHERE release_date > '2000-01-01' AND budget > 100000000
  ORDER BY popularity DESC
  LIMIT 1;
  """)

  result = connection.execute(most_popular)
  print(result.fetchall())


[('Interstellar', 724.247784, '2014-11-05 00:00:00.000000', 165000000)]


In [40]:
with data.connect() as connection:
  not_english = text("""
  SELECT COUNT(*)
  FROM movies
  WHERE original_language <> 'en';
  """)

  result = connection.execute(not_english)
  print(result.fetchall())

[(298,)]


In [66]:
columns_gn = inspector.get_columns('genres')

colums_names_gn = [column['name'] for column in columns_gn]

print(colums_names_gn)

['genre_id', 'genre_name']


In [59]:
with data.connect() as connection:
  query = text("""
    SELECT *
    FROM genres
    WHERE genre_name LIKE '%Romance and Comedy%';
  """)

  result = connection.execute(query)
  print(result.fetchall())

[]


In [47]:
columns_pd = inspector.get_columns('productioncompanymap')

colums_names_pd = [column['name'] for column in columns_pd]

print(colums_names_pd)

['movie_id', 'production_company_id']


In [50]:
with data.connect() as connection:
  query = text("""
  SELECT COUNT(*)
  FROM productioncompanymap
  WHERE production_company_id = 3;
  """)

  result = connection.execute(query)
  print(result.fetchall())

[(16,)]


In [67]:
from sqlalchemy import create_engine, text

# Create a connection to the TMDB database
engine = create_engine('sqlite:///TMDB.db')

# Connect to the database
with engine.connect() as connection:
    # Define the SQL query to get the genre IDs for Romance and Comedy
    genre_ids_query = text("""
    SELECT genre_id, genre_name
    FROM genres
    WHERE genre_name IN ('Romance', 'Comedy');
    """)

    # Execute the query to get the genre IDs
    result = connection.execute(genre_ids_query).fetchall()
    genre_ids = {row[1]: row[0] for row in result}  # Access tuple elements by index

    romance_id = genre_ids.get('Romance')
    comedy_id = genre_ids.get('Comedy')

    if romance_id and comedy_id:
        # Define the SQL query to count movies that are both Romance and Comedy
        count_query = text(f"""
        SELECT COUNT(DISTINCT m.movie_id) AS count
        FROM movies m
        JOIN genremap gm1 ON m.movie_id = gm1.movie_id AND gm1.genre_id = :romance_id
        JOIN genremap gm2 ON m.movie_id = gm2.movie_id AND gm2.genre_id = :comedy_id;
        """)

        # Execute the query to count the movies
        result = connection.execute(count_query, {'romance_id': romance_id, 'comedy_id': comedy_id}).scalar()

        # Display the result
        print(f"Number of movies that are both Romance and Comedy: {result}")
    else:
        print("Could not find genre IDs for Romance and Comedy.")


Number of movies that are both Romance and Comedy: 484


In [76]:
column_l = inspector.get_columns('languagemap')

colums_names_l = [column['name'] for column in column_l]

print(colums_names_l)

['movie_id', 'iso_639_1']


In [80]:
from sqlalchemy import create_engine, text

# Create a connection to the TMDB database
engine = create_engine('sqlite:///TMDB.db')

# Connect to the database
with engine.connect() as connection:
    # Define the SQL query to get the genre ID for Action
    genre_id_query = text("""
    SELECT genre_id
    FROM genres
    WHERE genre_name = 'Action';
    """)

    # Execute the query to get the Action genre ID
    result = connection.execute(genre_id_query).fetchone()
    action_genre_id = result[0] if result else None

    # Define the SQL query to get the language ID for German
    language_id_query = text("""
    SELECT iso_639_1
    FROM languages
    WHERE language_name = 'Deutsch';
    """)

    # Execute the query to get the German language ID
    result = connection.execute(language_id_query).fetchone()
    german_language_id = result[0] if result else None

    if action_genre_id and german_language_id:
        # Define the SQL query to find the most popular action movie with some German
        most_popular_query = text(f"""
        SELECT m.title, m.popularity
        FROM movies m
        JOIN genremap gm ON m.movie_id = gm.movie_id AND gm.genre_id = :action_genre_id
        JOIN languagemap lm ON m.movie_id = lm.movie_id AND lm.iso_639_1 = :german_language_id
        ORDER BY m.popularity DESC
        LIMIT 1;
        """)

        # Execute the query to find the most popular movie
        result = connection.execute(most_popular_query, {'action_genre_id': action_genre_id, 'german_language_id': german_language_id}).fetchone()

        # Display the result
        if result:
            print(f"Most popular action movie with some German: {result[0]} (Popularity: {result[1]})")
        else:
            print("No action movie with some German found.")
    else:
        print("Could not find genre ID for Action or language ID for German.")


Most popular action movie with some German: Captain America: Civil War (Popularity: 198.372395)


In [72]:
with data.connect() as connection:
  query = text("""
    SELECT *
    FROM movies

  """)

  result = connection.execute(query)
  print(result.fetchall())


[(5, 'Four Rooms', '1995-12-09 00:00:00.000000', 4000000, None, 'en', 'Four Rooms', "It's Ted the Bellhop's first night on the job...and the hotel's very unusual guests are about to place him in some outrageous predicaments. It seems that this evening's room service is serving up one unbelievable happening after another.", 22.87623, 4300000.0, 98.0, 'Released', "Twelve outrageous guests. Four scandalous requests. And one lone bellhop, in his first day on the job, who's in for the wildest New year's Eve of his life.", 6.5, 530), (11, 'Star Wars', '1977-05-25 00:00:00.000000', 11000000, 'http://www.starwars.com/films/star-wars-episode-iv-a-new-hope', 'en', 'Star Wars', 'Princess Leia is captured and held hostage by the evil Imperial forces in their effort to take over the galactic Empire. Venturesome Luke Skywalker a ... (14 characters truncated) ... tain Han Solo team together with the loveable robot duo R2-D2 and C-3PO to rescue the beautiful princess and restore peace and justice in t

In [82]:
colums_act = inspector.get_columns('actors')

colums_names_act = [column['name'] for column in colums_act]

print(colums_names_act)

['actor_id', 'actor_name', 'gender']


In [83]:
with data.connect() as connection:
  query = text("""
    SELECT *
    FROM actors
    WHERE actor_name LIKE '%Tom Cruise%';
  """)

  result = connection.execute(query)
  print(result.fetchall())

[(500, 'Tom Cruise', 2)]


In [85]:
column_cst = inspector.get_columns('casts')

colums_names_cst = [column['name'] for column in column_cst]

print(colums_names_cst)

['movie_id', 'actor_id', 'characters']


In [90]:
with data.connect() as connection:
  query = text("""
    SELECT COUNT(*)
    FROM casts
    WHERE actor_id = 500 AND characters = 'Ethan Hunt';
  """)

  result = connection.execute(query)
  print(result.fetchall())
#

[(5,)]


In [41]:
with data.connect() as connection:
    # Create an inspector object
    inspector = inspect(data)

    # Get a list of all table names
    tables = inspector.get_table_names()

    # Print all table names
    print("Tables in the database:")
    for table in tables:
        print(table)

Tables in the database:
actors
casts
genremap
genres
keywordmap
keywords
languagemap
languages
movies
oscars
productioncompanies
productioncompanymap
productioncountries
productioncountrymap
sysdiagrams


In [91]:
columns_oscr = inspector.get_columns('oscars')

colums_names_oscr = [column['name'] for column in columns_oscr]

print(colums_names_oscr)


['year', 'award', 'winner', 'name', 'film']


In [94]:
with engine.connect() as connection:
    # Define the SQL query to find the number of Oscar nominations for Cate Blanchett
    nominations_query = text("""
    SELECT COUNT(*)
    FROM oscars
    WHERE name = 'Cate Blanchett';
    """)

    # Execute the query to get the count
    result = connection.execute(nominations_query).scalar()

    # Display the result
    print(f"Cate Blanchett was nominated for an Oscar {result} times.")


Cate Blanchett was nominated for an Oscar 7 times.


In [97]:
from sqlalchemy import create_engine, text

# Create a connection to the TMDB database
engine = create_engine('sqlite:///TMDB.db')

# Connect to the database
with engine.connect() as connection:
    # Define the SQL query to get the language IDs for Afrikaans and Zulu
    language_ids_query = text("""
    SELECT iso_639_1
    FROM languages
    WHERE language_name IN ('Afrikaans', 'Zulu');
    """)

    # Execute the query to get the language IDs
    result = connection.execute(language_ids_query).fetchall()
    language_ids = [row[0] for row in result]

    print("Language IDs for Afrikaans and Zulu:", language_ids)

    if language_ids:
        # Define the SQL query to count movies with Afrikaans or Zulu
        count_query = text(f"""
        SELECT COUNT(DISTINCT lm.movie_id) AS count
        FROM languagemap lm
        WHERE lm.iso_639_1 IN ({','.join([':lang'+str(i) for i in range(len(language_ids))])});
        """)

        # Create a dictionary for the language IDs
        params = {f'lang{i}': lang_id for i, lang_id in enumerate(language_ids)}

        # Execute the query to count the movies
        result = connection.execute(count_query, params).scalar()

        # Display the result
        print(f"Number of movies that contain at least one of the official South African languages (Afrikaans or Zulu): {result}")
    else:
        print("Could not find language IDs for Afrikaans or Zulu.")


Language IDs for Afrikaans and Zulu: ['af']
Number of movies that contain at least one of the official South African languages (Afrikaans or Zulu): 7


In [98]:
with data.connect() as connection:
  query = text("""
    UPDATE languages
    SET language_name = 'Chinese'
    WHERE iso_639_1 = 'zh';
  """)

  print("operation complete")

operation complete


In [100]:
with data.connect() as connection:
  query = text("""
    INSERT INTO genres (genre_id, genre_name) VALUES (10, 'Sport');
  """)
  print("operation complete")

operation complete


In [102]:
with data.connect() as connection:
  query = text("""
    SELECT * FROM movies ORDER BY release_date ASC LIMIT 10
  """)
  result = connection.execute(query)
  print(result.fetchall())


[(380097, 'America Is Still the Place', None, 0, None, 'en', 'America Is Still the Place', '"1971 post civil rights San Francisco seemed like the perfect place for a black Korean War veteran and his family to realize their dream of economic  ... (222 characters truncated) ... pened a door that had never been open to a black man before. This is a story about what happened when he stepped through that door, with both feet!."', 0.0, 0.0, 0.0, 'Released', None, 0.0, 0), (3059, 'Intolerance', '1916-09-04 00:00:00.000000', 385907, None, 'en', 'Intolerance', 'The story of a poor young woman, separated by prejudice from her husband and baby, is interwoven with tales of intolerance from throughout history.', 3.232447, 8394751.0, 197.0, 'Released', 'The Cruel Hand of Intolerance', 7.4, 60), (3060, 'The Big Parade', '1925-11-05 00:00:00.000000', 245000, None, 'en', 'The Big Parade', "The story of an idle rich boy who joins the US Army's Rainbow Division and is sent to France to fight in World War